In [ ]:
!pip install python-Levenshtein

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import Levenshtein

In [ ]:
df = pd.read_csv('AptaBench_dataset_v2.csv')

print('Ознакомление с датасетом:')
print(f'Всего {len(df)} аптамеров')
print(f'Всего {df["sequence"].nunique()} уникальных последовательностей')
print(f'Всего {df["canonical_smiles"].nunique()} уникальных мишеней')
print(f'Список колонок: {df.columns}')

In [ ]:
print(df.info())

# Почистим от дубликатов
df_clean = df.drop_duplicates(subset=['sequence', 'canonical_smiles']).reset_index(drop=True)
print(df_clean.info())

**В датасете находится:**

1) Столбец `type` - РНК или ДНК наш аптамер
2) Столбец `sequence` - сами аптамеры
3) Столбец `canonical_smiles` - химическая структура мишени
4) Столбец `pKd_value` - показатель аффинности
5) Столбец `label` - бинарная классификация связывания
6) Столбец `buffer` - показатели химической среды
7) Столбец `origin` - источники/статьи
8) Столбец `source` - название датасетов-источников

In [ ]:
# Отбор только связанных с мишенью аптамеров
df_active = df_clean[df_clean['label'] == 1].copy()
print(df_active.info())

In [ ]:
# Убираем пустые значения pKd и фильтруем по порогу
df_pkd = df_active.dropna(subset=['pKd_value']).copy()
threshold = 6
df_final = df_pkd[df_pkd['pKd_value'] > threshold].copy()
print(df_final.info())

In [ ]:
print(f'Со сколькими мишенями связаны оставшиеся аптамеры: {df_final["canonical_smiles"].nunique()}')

df_final['seq_len'] = df_final['sequence'].str.len()
print(f'Смотрим разброс длин аптамеров:')
print(df_final['seq_len'].describe())

In [ ]:
print('По итогам очистки датасета:')
print(f'Всего {len(df_final)} аптамеров')
print(f'Всего {df_final["sequence"].nunique()} уникальных последовательностей')
print(f'Всего {df_final["canonical_smiles"].nunique()} уникальных мишеней')

Для повышения достоверности анализа скаффолдов была проведена фильтрация базы: удалены дубликаты аптамеров с такими же мишенями, исключены записи с низким сродством ($pKd < 6$), отсутствующими первоисточниками и условиями среды.